In [1]:
from glob import glob
from astroquery.mast import Mast, Observations
import os
import shutil
import numpy as np
import json
import requests
import asdf
from astropy import log
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table

from align_to_catalogs import *
import astropy.units as u

In [2]:
#realign_to_catalog(reference_coordinates, filtername='f212n',
#                       module='nrca',
#                       basepath='/orange/adamginsburg/jwst/brick/',
#                       fieldnumber='001',
#                       max_offset=0.4*u.arcsec,
#                       mag_limit=15,
#                       catfile=None, imfile=None)
#
#

In [3]:
0 + 1*u.arcsec

<Quantity 1. arcsec>

In [4]:
fov_regname = {'brick': 'regions/nircam_brick_fov.reg',
              'cloudc': 'regions/nircam_cloudc_fov.reg',
              }
regionname = 'cloudc'
basepath = f'/orange/adamginsburg/jwst/{regionname}/'
filtername = 'F405N'
field = '002' 
module = 'merged'
output_dir = f'/orange/adamginsburg/jwst/{regionname}/{filtername}/pipeline/'

In [5]:
imfile = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak.fits'
imfile_realigned = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak_realigned_to_vvv.fits'
shutil.copy(imfile, imfile_realigned)

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak_realigned_to_vvv.fits'

In [6]:
reference_catalog = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.fits'

In [7]:
reftbl = Table.read(reference_catalog)

In [8]:
reference_coordinates = reftbl['skycoord']

In [9]:
reftbl['skycoord']

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    [(266.581129, -28.666482), (266.594517, -28.666434),
     (266.604305, -28.663877), ..., (266.555803, -28.513283),
     (266.560494, -28.514834), (266.559398, -28.51401 )]>

In [10]:
cat_file = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv'
cat = Table.read(cat_file)

In [15]:
#cat['xcentroid'], cat['ycentroid']

In [12]:
ww =  WCS(fits.getheader(imfile, ext=('SCI', 1)))
ww

Set DATE-AVG to '2023-04-25T04:14:32.839' from MJD-AVG.
Set DATE-END to '2023-04-25T04:14:59.681' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -11.485975 from OBSGEO-[XYZ].
Set OBSGEO-H to 1386082407.725 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 266.59237008461  -28.557172894513  
CRPIX : 1024.5  1024.5  
CD1_1 CD1_2  : -1.6126880967505e-06  -1.7440453913269e-05  
CD2_1 CD2_2  : -1.7358691201541e-05  1.6484453417233e-06  
NAXIS : 2048  2048

In [13]:
skycrds_cat = ww.pixel_to_world(cat['xcentroid'], cat['ycentroid'])
skycrds_cat

<SkyCoord (ICRS): (ra, dec) in deg
    [(266.6116505 , -28.56882209), (266.61113913, -28.57424765),
     (266.61090323, -28.57649018), ..., (266.54867726, -28.65584137),
     (266.54533313, -28.67274956), (266.54349879, -28.68148105)]>

In [16]:
ww.wcs.crval

array([266.59237008, -28.55717289])

In [17]:
#ww.wcs.crval = ww.wcs.crval - [(0.1*u.arcsec).to(u.deg).value, (0.1*u.arcsec).to(u.deg).value]
#ww.wcs.crval

In [18]:
#realign_to_catalog(reference_coordinates=reference_coordinates, filtername=filtername.lower(), module=module, 
#                   basepath=basepath, fieldnumber=field, imfile=imfile_realigned, decoffset=-8*u.arcsec)

In [19]:
from jwst.scripts import asn_edit

In [19]:
asn_edit??

Type:        module
String form: <module 'jwst.scripts.asn_edit' from '/blue/adamginsburg/adamginsburg/repos/jwst/jwst/scripts/asn_edit.py'>
File:        /blue/adamginsburg/adamginsburg/repos/jwst/jwst/scripts/asn_edit.py
Source:     
#!/usr/bin/env python

import argparse

from jwst.associations import asn_edit


def main():
    """
    Parse command line, read, edit, write association file
    """

    # Parse command line arguments
    description_text = """
Edit Association File

This script adds or removes filenames from an association file. The
first argument is the name of the association file. Subsequent
arguments are the filenames. Options determine which operation is
performed: --add or --remove. If adding files the --type option sets
the exposure type for the new files. If removing file, the --ignore
option will not use the filename suffix when matching the filenames for
removal. Normally the output association file is the same as the input.
The --output option allows you to

In [20]:
shutil.copy('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn.json', 
            '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json')

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json'

In [22]:
asn_edit('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json')

TypeError: 'module' object is not callable

In [ ]:
#realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field,
#                                   imfile=realigned_vvv_filename, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15)

In [23]:
ims = glob(os.path.join(output_dir, f'*nrca*_destreak.fits'))
ims

['/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00003_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00004_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00003_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00001_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00004_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00003_nrcalong_destre

In [24]:
i = ims[0]
i.split('.')[0]+'_realigned.fits'

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak_realigned.fits'

In [25]:
for i in ims:
    realigned_vvv_i = i.split('.')[0]+'_realigned.fits'
    shutil.copy(i, realigned_vvv_i)
    realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field, 
                   imfile=realigned_vvv_i, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15, decoffset=-8*u.arcsec)

2023-07-28 16:55:30,240 - stpipe - INFO - Kept 7752 out of 9945 VVV stars using ksmag_limit>11


INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]


2023-07-28 16:55:31,588 - stpipe - INFO - For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15
2023-07-28 16:55:31,689 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/repos/astropy/astropy/wcs/wcs.py:2966: AstropyWarning: Some non-standard WCS keywords were excluded: A_ORDER, A_0_2, A_0_3, A_1_1, A_1_2, A_2_0, A_2_1, A_3_0, B_ORDER, B_0_2, B_0_3, B_1_1, B_1_2, B_2_0, B_2_1, B_3_0, AP_ORDER, AP_0_2, AP_0_3, AP_1_1, AP_1_2, AP_2_0, AP_2_1, AP_3_0, BP_ORDER, BP_0_2, BP_0_3, BP_1_1, BP_1_2, BP_2_0, BP_2_1, BP_3_0 Use the ``relax`` kwarg to control this.
  warnings.warn(

2023-07-28 16:55:31,721 - stpipe - INFO - 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion

INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-07-28T16:55:31.679563: Before realignment, offset is -0.012222526811456191 arcsec, -0.0226795682010561 arcsec.  Found 309 matches.
2023-07-28T16:55:31.685560: CRVAL before 266.59541974652 -28.538974425935
2023-07-28T16:55:31.694481: CRVAL after 266.59542314167 -28.536745903833
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
   

2023-07-28 16:55:32,281 - stpipe - INFO - Kept 7752 out of 9945 VVV stars using ksmag_limit>11


...

INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-07-28T16:56:02.574818: Before realignment, offset is -0.00997715246739972 arcsec, 0.010978262529448557 arcsec.  Found 200 matches.
2023-07-28T16:56:02.580819: CRVAL before 266.58377837707 -28.593542796145
2023-07-28T16:56:02.590249: CRVAL after 266.5837811485 -28.59132362344
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
     

2023-07-28 16:56:03,163 - stpipe - INFO - Kept 7752 out of 9945 VVV stars using ksmag_limit>11


INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]


2023-07-28 16:56:04,486 - stpipe - INFO - For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15
2023-07-28 16:56:04,583 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/repos/astropy/astropy/wcs/wcs.py:2966: AstropyWarning: Some non-standard WCS keywords were excluded: A_ORDER, A_0_2, A_0_3, A_1_1, A_1_2, A_2_0, A_2_1, A_3_0, B_ORDER, B_0_2, B_0_3, B_1_1, B_1_2, B_2_0, B_2_1, B_3_0, AP_ORDER, AP_0_2, AP_0_3, AP_1_1, AP_1_2, AP_2_0, AP_2_1, AP_3_0, BP_ORDER, BP_0_2, BP_0_3, BP_1_1, BP_1_2, BP_2_0, BP_2_1, BP_3_0 Use the ``relax`` kwarg to control this.
  warnings.warn(

2023-07-28 16:56:04,614 - stpipe - INFO - 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion

INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-07-28T16:56:04.572775: Before realignment, offset is -0.0013438789665087825 arcsec, -0.04238846712354416 arcsec.  Found 229 matches.
2023-07-28T16:56:04.578843: CRVAL before 266.59154983035 -28.556436704257
2023-07-28T16:56:04.588157: CRVAL after 266.59155020365 -28.554202707461
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
 